In [1]:
!pip install torchmetrics datasets tokenizers

In [43]:
from datasets import load_dataset
wmt14 = load_dataset('wmt14', 'de-en')

Found cached dataset wmt14 (/home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
!git clone https://github.com/hynky1999/Statistical-learning-class
%cd /content/Statistical-learning-class/Assigments/Project

Cloning into 'Statistical-learning-class'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 206 (delta 54), reused 177 (delta 29), pack-reused 0
Receiving objects: 100% (206/206), 1.58 MiB | 5.23 MiB/s, done.
Resolving deltas: 100% (54/54), done.
[Errno 2] No such file or directory: '/content/Statistical-learning-class/Assigments/Project'
/home/kydliceh/Projects/R/Assigments/Project


In [44]:
train_subset_length = 10
test_subset_length = 2000
vocab_size=10

In [45]:
train_dataset = wmt14['train'].select(range(train_subset_length))
test_dataset = wmt14['test'].select(range(test_subset_length))

In [64]:
class ProcessorSequence(processors.PostProcessor):
    def __init__(self, processors):
        self.processors = processors
    
    def num_special_tokens_to_add(self, pair=False):
        return sum(p.num_special_tokens_to_add(pair) for p in self.processors)

    def process(self, encoding, pair=None, add_special_tokens=True):
        for p in self.processors:
            encoding = p.process(encoding, pair, add_special_tokens)
        return encoding

TypeError: type 'tokenizers.processors.PostProcessor' is not an acceptable base type

In [90]:
from tokenizers import ByteLevelBPETokenizer



def create_tokenizer(iterable, add_special_tokens=False, train=True, vocab_size=1000):
    toknizer = ByteLevelBPETokenizer()
    if train:
        toknizer.train_from_iterator(iterable, vocab_size=vocab_size, min_frequency=2, special_tokens=["[PAD]"])

    if add_special_tokens:
        toknizer.add_special_tokens(["[START]", "[END]"])
    return toknizer


    

In [91]:

de_it = map(lambda x: x['de'] , train_dataset['translation'])
en_it = map(lambda x: x['en'] , train_dataset['translation'])

In [49]:
de_token = de_token.from_file("de_token.json")
en_token = en_token.from_file("en_token.json")

NameError: name 'de_token' is not defined

In [92]:
de_token = create_tokenizer(de_it, train=True, vocab_size=vocab_size, add_special_tokens=True)
en_token = create_tokenizer(en_it, train=True, vocab_size=vocab_size)


In [34]:
de_token.save("de_token.json")
en_token.save("en_token.json")

In [100]:
import torch
import numpy as np

In [122]:
from torch.utils.data import DataLoader
def extract_embedding(embeds, lang):
    return {f"{lang}_ids": [e.ids for e in embeds], f"{lang}_att": [e.attention_mask for e in embeds]}

def add_lenghts(trans):
  translation = trans["translation"]
  de_sent = [t["de"] for t in translation]
  en_sent = [t["en"] for t in translation]
  en_len = [len(t["en"]) for t in translation]
  de_len = [len(t["de"]) for t in translation]
  return {"en_len": en_len, "de_len": de_len,"de_sent": de_sent, "en_sent": en_sent}



def tokenize(trans):
    translation = trans["translation"]
    de_sent = [t["de"] for t in translation]
    en_sent = [t["en"] for t in translation]
    en = en_token.encode_batch(en_sent)
    de = de_token.encode_batch(de_sent)
    dct = {**extract_embedding(en, "en"), **extract_embedding(de, "de")}
    return dct

def collate_fc(batch):
    en_ids_padded = torch.nn.utils.rnn.pad_sequence([b["en_ids"] for b in batch], batch_first=True, padding_value=en_token.token_to_id("[PAD]"))

    de_ids = [torch.cat([torch.tensor([de_token.token_to_id("[START]")]), b["de_ids"], torch.tensor([de_token.token_to_id("[END]")])]) for b in batch]

    de_ids_padded = torch.nn.utils.rnn.pad_sequence(de_ids, batch_first=True, padding_value=de_token.token_to_id("[PAD]"))

    en_sents = [b["en_sent"] for b in batch]
    de_sents = [b["de_sent"] for b in batch]

    return {"en_ids": en_ids_padded, "de_ids": de_ids_padded, "de_sent": de_sents, "en_sent": en_sents}

def create_dataloader(dataset, batch_size=32, shuffle=False):
    tokenized = dataset.map(add_lenghts, batch_size=batch_size, batched=True)
    # Sort by lengths to get smaller paddings
    tokenized = tokenized.sort("en_len")
    tokenized = tokenized.sort("de_len",kind="stable")
    tokenized = tokenized.map(tokenize, batch_size=batch_size, batched=True)
    tokenized = tokenized.remove_columns("translation")
    tokenized.set_format("torch")
    return DataLoader(tokenized, batch_size=batch_size, shuffle=True,collate_fn=collate_fc)



In [123]:
de_token.token_to_id("[END]")

258

In [126]:
dataloader_train = create_dataloader(train_dataset, batch_size=32, shuffle=True)
dataloader_test = create_dataloader(test_dataset, batch_size=32, shuffle=True)

Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-865161eab3354bb8.arrow
Loading cached sorted indices for dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-65228b20db78c273.arrow
Loading cached sorted indices for dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-61ec2f49f1cec7c6.arrow
Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-51df25cc70a98ecc.arrow
Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-593bbaf8b7994ff6.arrow
Loading cached sorted indic

In [14]:
%load_ext autoreload
%autoreload 2

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
from train_test import train, evaluate
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

In [17]:
from model import WMTModel
d_model=128
model = WMTModel(en_token.get_vocab_size(), de_token.get_vocab_size() , d_model)
# Set to square root of model
# Then multiply by  min(step_num^−0.5 , step_num * warmup_steps^−1.5)
initial_lr = d_model ** -0.5
warmup_steps = 4000
multiplier_lambda = lambda step: min((step+1) ** -0.5, (step+1) * warmup_steps ** -1.5)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-9, lr=initial_lr)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, multiplier_lambda)
criterion = nn.CrossEntropyLoss(ignore_index=0)
writer = SummaryWriter()
epochs=10
save_every=2
for epoch in range(epochs):
    train(model, optimizer, scheduler ,criterion, dataloader_train, writer, epoch, minibatch=False)

    if epoch % save_every == 0:
        torch.save(model.state_dict(), f"model_{epoch}.pt")


2023-01-02 09:20:53.743446: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 09:20:54.830852: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-02 09:20:54.830879: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-02 09:20:58.077854: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

ZeroDivisionError: integer division or modulo by zero

In [ ]:
evaluate(model, dataloader_test, writer, de_token)